In [1]:
pip install pynini

In [2]:
# Fonction utilitaire pour convertir les nombres en lettres (0-1000, version corrigée)
def number_to_words_fr(n):
    base = [
        "zéro", "un", "deux", "trois", "quatre", "cinq", "six", "sept", "huit", "neuf",
        "dix", "onze", "douze", "treize", "quatorze", "quinze", "seize", "dix-sept", "dix-huit", "dix-neuf", "vingt"
    ]
    dizaines = ["", "", "vingt", "trente", "quarante", "cinquante", "soixante", "soixante-dix", "quatre-vingt", "quatre-vingt-dix"]
    if n <= 20:
        return base[n]
    elif n < 70:
        d, u = divmod(n, 10)
        if u == 1 and d != 8:
            return dizaines[d] + "-et-un"
        elif u == 0:
            return dizaines[d]
        else:
            return dizaines[d] + "-" + base[u]
    elif n < 80:
        # 70-79
        return "soixante-" + number_to_words_fr(n-60)
    elif n < 100:
        # 80-99
        if n == 80:
            return "quatre-vingts"
        else:
            return "quatre-vingt-" + number_to_words_fr(n-80)
    elif n == 100:
        return "cent"
    elif n < 200:
        if n == 100:
            return "cent"
        else:
            return "cent " + number_to_words_fr(n-100)
    elif n < 1000:
        c, r = divmod(n, 100)
        centaine = base[c] + " cent"
        if r == 0:
            if c > 1:
                centaine += "s"
            return centaine
        else:
            return centaine + " " + number_to_words_fr(r)
    elif n == 1000:
        return "mille"
    else:
        return str(n)

In [3]:
# Test contextuel : normalisation sur des phrases entières
import re
import pynini
from pynini.lib import pynutil

def chiffre_map_0_59():
    chiffres = [
        "zéro", "un", "deux", "trois", "quatre", "cinq", "six", "sept", "huit", "neuf",
        "dix", "onze", "douze", "treize", "quatorze", "quinze", "seize", "dix-sept",
        "dix-huit", "dix-neuf", "vingt", "vingt-et-un", "vingt-deux", "vingt-trois", "vingt-quatre",
        "vingt-cinq", "vingt-six", "vingt-sept", "vingt-huit", "vingt-neuf", "trente", 
        "trente-et-un", "trente-deux", "trente-trois", "trente-quatre", "trente-cinq",
        "trente-six", "trente-sept", "trente-huit", "trente-neuf", "quarante", "quarante-et-un",
        "quarante-deux", "quarante-trois", "quarante-quatre", "quarante-cinq", "quarante-six",
        "quarante-sept", "quarante-huit", "quarante-neuf", "cinquante", "cinquante-et-un",
        "cinquante-deux", "cinquante-trois", "cinquante-quatre", "cinquante-cinq",
        "cinquante-six", "cinquante-sept", "cinquante-huit", "cinquante-neuf"
    ]
    return pynini.string_map([(str(i).zfill(2), chiffres[i]) for i in range(60)])

def heure_map_24():
    heures = [
        "minuit", "une", "deux", "trois", "quatre", "cinq", "six", "sept", "huit", "neuf",
        "dix", "onze", "douze", "treize", "quatorze", "quinze", "seize", "dix-sept",
        "dix-huit", "dix-neuf", "vingt", "vingt-et-un", "vingt-deux", "vingt-trois"
    ]
    return pynini.string_map([(str(i).zfill(2), heures[i]) for i in range(24)])

def build_fst_heures_lettres():
    heure = heure_map_24()
    minute = chiffre_map_0_59()
    # "12:47" -> "douze quarante-sept"
    # Correction : cross sur chaque composant
    pattern = heure + pynini.accep(":") + minute
    pattern = pynini.cdrewrite(pynini.cross(":", " "), "", "", "") @ pattern
    return pattern.optimize()

def build_fst_argent_lettres():
    chiffres = [
        "zéro", "un", "deux", "trois", "quatre", "cinq", "six", "sept", "huit", "neuf",
        "dix", "onze", "douze", "treize", "quatorze", "quinze", "seize", "dix-sept",
        "dix-huit", "dix-neuf", "vingt", "vingt-et-un", "vingt-deux", "vingt-trois", "vingt-quatre",
        "vingt-cinq", "vingt-six", "vingt-sept", "vingt-huit", "vingt-neuf", "trente", 
        "trente-et-un", "trente-deux", "trente-trois", "trente-quatre", "trente-cinq",
        "trente-six", "trente-sept", "trente-huit", "trente-neuf", "quarante", "quarante-et-un",
        "quarante-deux", "quarante-trois", "quarante-quatre", "quarante-cinq", "quarante-six",
        "quarante-sept", "quarante-huit", "quarante-neuf", "cinquante", "cinquante-et-un",
        "cinquante-deux", "cinquante-trois", "cinquante-quatre", "cinquante-cinq",
        "cinquante-six", "cinquante-sept", "cinquante-huit", "cinquante-neuf"
    ]
    int_map = pynini.string_map([(str(i).zfill(2), chiffres[i]) for i in range(60)])
    devise = pynini.string_map([
        ("$", "dollars"),
        ("€", "euros")
    ])
    # Correction : séparer les montants et centimes
    montant = int_map + pynini.accep(",") + int_map + pynutil.insert(" dollars et ") + int_map + pynutil.insert(" cents")
    montant_euro = int_map + pynini.accep(",") + int_map + pynutil.insert(" euros et ") + int_map + pynutil.insert(" cents")
    montant_simple = int_map + pynutil.insert(" ") + devise
    pattern = montant | montant_euro | montant_simple
    return pattern.optimize()

def build_cardinal_fst_fr():
    from __main__ import number_to_words_fr
    union = None
    for n in range(0, 1001):
        words = number_to_words_fr(n)
        pair = pynini.cross(str(n), words)
        if union is None:
            union = pair
        else:
            union |= pair
    return pynini.optimize(union)

def build_fst_unite_fr():
    cardinal = build_cardinal_fst_fr()
    unite = pynini.string_map([
        ("kg", "kilogrammes"), ("g", "grammes"), ("m", "mètres"), ("cm", "centimètres"),
        ("mm", "millimètres"), ("km", "kilomètres"), ("L", "litres"), ("ml", "millilitres"),
        ("h", "heures"), ("min", "minutes"), ("s", "secondes")
    ])
    nombre = cardinal | (cardinal + pynini.accep(",") + cardinal)
    pattern = nombre + pynutil.insert(" ") + unite
    return pattern.optimize()

fst_heure = build_fst_heures_lettres()
fst_argent = build_fst_argent_lettres()
fst_cardinal = build_cardinal_fst_fr()
fst_unite = build_fst_unite_fr()
chiffre_fst = chiffre_map_0_59()

def _replace(match):
    s = match.group(0)
    # Heures
    if re.match(r"^\d{2}:\d{2}$", s):
        try:
            # Correction : split et cross séparément
            h, m = s.split(":")
            heure_txt = pynini.shortestpath(pynini.accep(h.zfill(2), token_type="utf8") @ heure_map_24()).string("utf8")
            minute_txt = pynini.shortestpath(pynini.accep(m.zfill(2), token_type="utf8") @ chiffre_map_0_59()).string("utf8")
            return f"{heure_txt} {minute_txt}"
        except Exception as e:
            return f"[erreur heure: {e}]"
    # Montant
    if re.match(r"^\d{2},\d{2} ?[$€]$", s):
        try:
            return pynini.shortestpath(pynini.accep(s, token_type="utf8") @ fst_argent).string("utf8")
        except Exception as e:
            return f"[erreur argent: {e}]"
    if re.match(r"^\d{2} ?[$€]$", s):
        try:
            return pynini.shortestpath(pynini.accep(s[:2], token_type="utf8") @ chiffre_fst).string("utf8") + " " + ("euros" if "€" in s else "dollars")
        except Exception as e:
            return f"[erreur argent simple: {e}]"
    # Unités
    if re.match(r"^\d{1,4}(,\d{1,4})? ?(kg|g|m|cm|mm|km|L|ml|h|min|s)$", s):
        try:
            return pynini.shortestpath(pynini.accep(s.replace(" ", ""), token_type="utf8") @ fst_unite).string("utf8")
        except Exception as e:
            return f"[erreur unité: {e}]"
    # Cardinal 0-1000
    if s.isdigit() and 0 <= int(s) <= 1000:
        try:
            return pynini.shortestpath(pynini.accep(s, token_type="utf8") @ fst_cardinal).string("utf8")
        except Exception as e:
            return f"[erreur cardinal: {e}]"
    return s

pattern = r"\d{2}:\d{2}|\d{2},\d{2} ?[$€]|\d{2} ?[$€]|\b\d{1,4}(,\d{1,4})? ?(kg|g|m|cm|mm|km|L|ml|h|min|s)\b|\b\d{1,4}\b"


In [4]:
phrases = [
    "Il est 10:54.",
    "J'ai 8,45 $.",
    "Il a 7 chiens.",
    "Le poids est 5 kg.",
    "La distance est 12 m.",
    "Il est 00:01.",
    "J'ai 1000 euros.",
    "3,50 €",
    "Il a 100 chats.",
    "Le nombre est 1000."
]

for phrase in phrases:
    print(f"Entrée : {phrase} => Normalisé : {re.sub(pattern, _replace, phrase)}")


Entrée : Il est 10:54. => Normalisé : Il est dix cinquante-quatre.
Entrée : J'ai 8,45 $. => Normalisé : J'ai huit,quarante-cinq dollars.
Entrée : Il a 7 chiens. => Normalisé : Il a sept chiens.
Entrée : Le poids est 5 kg. => Normalisé : Le poids est cinq kilogrammes.
Entrée : La distance est 12 m. => Normalisé : La distance est douze mÃ¨tres.
Entrée : Il est 00:01. => Normalisé : Il est minuit un.
Entrée : J'ai 1000 euros. => Normalisé : J'ai mille euros.
Entrée : 3,50 € => Normalisé : trois,cinquante euros
Entrée : Il a 100 chats. => Normalisé : Il a cent chats.
Entrée : Le nombre est 1000. => Normalisé : Le nombre est mille.


In [6]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.1 MB/s eta 0:00:0000:0100:01


In [ ]:
# Utilisation d'un package dédié pour le calcul du WER et lecture du fichier test_cases_cardinal_fr.txt
import jiwer

# Lire les 14 premières lignes du fichier test_cases_cardinal_fr.txt
refs = []
hyps = []

with open("test_cases_cardinal_fr.txt", encoding="utf8") as f:
    for i, line in enumerate(f):
        if i >= 14:
            break
        if "~" in line:
            nombre, reference = line.strip().split("~", 1)
            refs.append(reference)
            # Hypothèse = sortie du système de normalisation sur l'entrée 'nombre'
            # On applique la normalisation à l'entrée 'nombre'
            # On retire les espaces internes pour les milliers:
            nombre_normalise = nombre.replace(" ", "")
            hyp = re.sub(pattern, _replace, nombre_normalise)
            hyps.append(hyp)

# Calcule le WER avec jiwer
wer = jiwer.wer(refs, hyps)

for ref, hyp in zip(refs, hyps):
    print(f"REF: {ref}\nHYP: {hyp}\nWER: {jiwer.wer(ref, hyp):.3f}\n")

print(f"WER moyen sur les 14 premiers cas: {wer:.3f}")

REF : Il est dix heures cinquante-quatre .
HYP : Il est dix cinquante-quatre.
WER : 0.500

REF : J'ai huit dollars quarante-cinq .
HYP : J'ai huit,quarante-cinq dollars.
WER : 0.800

REF : Il a sept chiens .
HYP : Il a sept chiens.
WER : 0.400

REF : Le poids est cinq kilogrammes .
HYP : Le poids est cinq kilogrammes.
WER : 0.333

REF : La distance est douze mètres .
HYP : La distance est douze mÃ¨tres.
WER : 0.333

REF : Il est zéro heure une .
HYP : Il est minuit un.
WER : 0.667

REF : J'ai mille euros .
HYP : J'ai mille euros.
WER : 0.500

REF : trois euros cinquante
HYP : trois,cinquante euros
WER : 0.667

REF : Il a cent chats .
HYP : Il a cent chats.
WER : 0.400

REF : Le nombre est mille .
HYP : Le nombre est mille.
WER : 0.400

WER moyen sur l'ensemble des phrases : 0.500
